# Import some stuff

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Retrieve the data

In [2]:
data = pd.read_csv('fifa21_train.csv')
data.shape

(11701, 101)

# Start the cleaning

## Quinn

In [ ]:
Weight = [] 
for i in data['Weight']:
    Weight.append(i.replace('lbs' , ''))

data['Weight'] = Weight
data['Weight'] = data['Weight'].astype(np.number)
data['Weight'] = data['Weight']/2.2046

## Remove redundant columns

In [3]:
# Column 'Team & Contract' can go, because it only combines values for two other columns.
data = data.drop(columns=['Team & Contract'])

# Next, there are a bunch of columns whose values are factored into other columns (like 'Attacking', 'Movement' etc)
todrop = ['Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'Standing Tackle', 'Sliding Tackle', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes']
data = data.drop(columns=todrop, axis = 1)



## Transform the 'Contract' column to only give the end date of the contract 

In [4]:
contractend = []
for item in data['Contract']:
    item = item.replace(' On Loan', '')
    contractend.append(item[-4:])

data['Contract']=contractend



## Remove stars

In [5]:
def removestar(dataframe, column):
    newvalues = []
    for item in dataframe[column]:
        item = item.replace('★', '')
        item = item.strip()
        newvalues.append(int(item))
    dataframe[column]=newvalues
    return dataframe


removestar(data,'IR')
removestar(data,'W/F')
removestar(data,'SM')

,ID,Name,Age,Nationality,Club,BP,Position,Height,Weight,foot,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,"5'9""",161lbs,Right,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"6'0""",159lbs,Right,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,"5'4""",134lbs,Right,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,"5'10""",152lbs,Right,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,"5'11""",150lbs,Right,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,"6'1""",168lbs,Left,...,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2,65
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,"6'1""",174lbs,Right,...,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2,70
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,"5'6""",143lbs,Right,...,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2,67
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,NaN,"6'2""",179lbs,Right,...,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68


## Transform the 'loan date end' column to a loan (yes/no) column

In [6]:
data['Loan Date End'] = data['Loan Date End'].fillna(0)

In [7]:
data['Loan Date End'] = data['Loan Date End'].fillna(0)

loanyesno = []
for item in data['Loan Date End']:
    if item == 0:
        loanyesno.append(item)
    else: 
        loanyesno.append(1)

data['Loan Date End'] = loanyesno
data = data.rename(columns={'Loan Date End':'On Loan'})

data['On Loan']

0        0
1        1
2        0
3        0
4        0
        ..
11696    0
11697    0
11698    0
11699    0
11700    0
Name: On Loan, Length: 11701, dtype: int64

## Remove the plusses and minuses on the last few rows

In [8]:
# Remove the plusses and minuses on the last few rows

toclean = ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK']

superlist = []

for column in toclean: 
    values = []
    for item in data[column]:
        if item[1] == '-' or item[1] == '+':
            values.append(int(item[0]))
        else: 
            values.append(int(item[:2]))
    superlist.append(values)


for i in range(len(toclean)):
    data[toclean[i]] = superlist[i]


## Convert height to cm

In [9]:
cmheight = []

for item in data['Height']:
    item = item.replace('\"', '')
    feetinches = item.split('\'')
    cmheight.append(round((int(feetinches[0]) * 30.48) + (int(feetinches[1]) * 2.54), 0))

In [10]:
data['Height'] = cmheight

## Clean up currency cells

In [11]:
def cleancurrency(dataframe, column):
    newvalues = []
    for item in dataframe[column]:
        item = item.replace('€', '')
        if 'K' in item: 
            item = float(item.replace('K', ''))
            item *= 1000
            newvalues.append(int(item))
        elif 'M' in item: 
            item = float(item.replace('M', ''))
            item *= 1000000      
            newvalues.append(int(item))
        else: 
            item = int(item)
            newvalues.append(int(item))
    dataframe[column]=newvalues
    return dataframe

cleancurrency(data, 'Value')
cleancurrency(data, 'Wage')
cleancurrency(data, 'Release Clause')


data['Value']

0         525000
1        8500000
2        9000000
3         275000
4         725000
          ...   
11696     625000
11697    1600000
11698    1100000
11699          0
11700     700000
Name: Value, Length: 11701, dtype: int64

## Clean up Hits column (which contains some values with K in them)

In [13]:
newhits = []
for item in data['Hits']:
    if 'K' in item:
        item = item.replace('K','')
        item = float(item) * 1000
        newhits.append(int(item))
    else: 
        newhits.append(int(item))
data['Hits'] = newhits


In [14]:
for item in data['Hits']:
    if not str(item).isnumeric(): 
        print(item)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11701 entries, 0 to 11700
Data columns (total 66 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              11701 non-null  int64  
 1   Name            11701 non-null  object 
 2   Age             11701 non-null  int64  
 3   Nationality     11701 non-null  object 
 4   Club            11683 non-null  object 
 5   BP              11701 non-null  object 
 6   Position        11413 non-null  object 
 7   Height          11701 non-null  float64
 8   Weight          11701 non-null  object 
 9   foot            11701 non-null  object 
 10  Growth          11701 non-null  int64  
 11  Joined          11660 non-null  object 
 12  On Loan         11701 non-null  int64  
 13  Value           11701 non-null  int64  
 14  Wage            11701 non-null  int64  
 15  Release Clause  11701 non-null  int64  
 16  Contract        11701 non-null  object 
 17  Attacking       11701 non-null 